<a href="https://colab.research.google.com/github/biggymart/colab/blob/main/ChaptchaRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.youtube.com/watch?v=IcLEJB2pY2Y&t=1052s

Title: Captcha recognition using PyTorch (Convolutional-RNN + CTC Loss)

Book: Approaching (Almost) Any Machine Learning Problem (Abhishek Thakur)

cf> for Keras version of it, refer to: https://keras.io/examples/vision/captcha_ocr/

모르는 부분 list-up:
1.   albumentation, .Compose
2.   np.transpose(2, 0, 1)
3.   glob
4.   list in a list (list comprehension)
5.   label encoder




In [ ]:
# config.py

DATA_DIR = '../input/captcha_images_v2'
BATCH_SIZE = 8
IMAGE_WIDTH = 300
IMAGE_HEIGHT = 75
NUM_WORKERS = 8
EPOCHS = 200
DEVICE = "cuda"

captcha 데이터셋을 보면, png 이미지들이 있음을 확인할 수 있음.

In [ ]:
# dataset.py

import albumentations # for augmentation
import torch
import numpy as np

from PIL import Image # to read image
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

class Classification:
  def __init__(self, image_paths, targets, resize=None):
    # resize: (h, w)
    self.image_paths = image_paths
    self.targets = targets
    self.resize = resize
    self.aug = albumentation.Compose([albumentations.Normalize(always_apply=True)])
  
  def __len__(self):
    return len(self.image_paths)

  # this part is different from the keras tutorial
  def __getitem__(self, item):
    image = Image.open(self.image_paths[item]).convert("RGB")
    targets = self.targets[item]

    if self.resize is not None:
      image = image.resize(
          (self.resize[1], self.resize[0]), resample=Image.BILINEAR
         )

    image = np.array(image)
    augmented = self.aug(image=image)
    image = augmented["image"]
    image = np.transpose(image, (2, 0, 1)).astype(np.float32)
    return {
        "images": torch.tensor(image, dtype=torch.float),
        "targets": torch.tensor(targets, dtype=torch.long)
    }

In [ ]:
# train.py

import os
import glob
import torch
import numpy as np

from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

import config
import dataset

# you can also grab all the images and create csv
# but it is not implemented here
def run_training():
  image_files = glob.glob(os.path.join(config.DATA_DIR, "*.png"))
  # "/../.../whatever.png" -> last element: "whatever.png" -> crop until last 4: "whatever"
  targets_orig = [x.split("/")[-1][:4] for x in image_files]
  # if your targets were "abcde" -> it becomes targets_flat = ['a', 'b', 'c', 'd', 'e']
  # list in a list
  targets = [[c for c in x] for x in targets_orig]
  targets_flat = [c for clist in targets for c in clist]

  lbl_enc = preprocessing.LabelEncoder()
  lbl_enc.fit(targets_flat)
  # we want to create an encoded target
  targets_enc = [lbl_enc.transform(x) for x in targets]
  targets_enc = np.array(targets_enc) + 1 # encoding starts with 0 and we want to keep 0 for unknown
  print(targets_enc)
  print(len(lbl_enc.classes_))
  print(np.unique(targets_flat))

  train_imgs, test_imgs, train_targets, test_targets, train_orig_targets, test_orig_targets = model_selection.train_test_split(image_files, targets_enc, targets_orig, test_size=0.1, random_state=42)
  train_dataset = dataset.ClassificationDataset(image_paths=train_imgs, targets=train_targets, resize(config.IMAGE_HEIGHT, config.IMAGE_WIDTH))


if __name__ -- "__main__":
  run_training()